In [1]:
from tutorialFunctions import *

# Tangent-space methods for uniform matrix product states
## 1. Matrix product states in the thermodynamic limit
### 1.1 Normalisation
We start by considering a uniform MPS in the thermodynamic limit, which is defined by 
$$ |\Psi(A)> = \sum_i^d \nu_L^\dagger \left[ \prod_{m\in Z} A_i \right]\nu_R |i>. $$

Here, $A_i$ are complex matrices of size $D \times D$, for every entry of the index $d$. This allows for the interpretation of the object $A$ as a three-legged tensor of dimensions $D\times d \times D$, where we will refer to $D$ as the bond dimension and $d$ as the physical dimension. With this object and the diagrammatic language of tensor networks, we can represent the state as

![image.png](img/MPSstate.png)

Thus, we initialise and represent a uniform MPS state as follows:

In [2]:
d = 3 # physical dimension
D = 5 # bond dimension
A = np.random.rand(D, d, D) + 1j*np.random.rand(D, d, D)

One of the most relevant objects in all our calculations will be the transfer matrix, defined as

![image.png](img/transferMatrix.png)

where we will be using the convention of ordering the legs as
1. top left
2. bottom left
3. top right
4. bottom right

The transfer matrix can be shown to be a completely positive map, such that the leading eigenvalue is a positive number, which we conveniently rescale to unity for a proper normalization of the state in the thermodynamic limit. This means solving the (left and right) eigenvalue equation:
![image.png](img/fixedPoints.png)

Additionally, we may fix the normalisation of the eigenvectors by requiring that their trace is equal to one:
![image.png](img/traceNorm.png)

In [3]:
E = np.einsum('isk,jsl->ijkl', A, np.conj(A))

"""numpy eigs provides efficient way to find largest eigenvalue/vector
    arg1 = matrix-like operator to find eigenvalue
    k = amount of eigenvalues/vectors to return
    which = 'LM' select largest magnitude eigenvalues"""

# right fixed point
lambdaRight, r = eigs(np.resize(E, (D**2, D**2)), k=1, which='LM')
r.resize(D,D)

# left fixed point (via transpose transfer matrix)
lambdaLeft, l = eigs(np.resize(E, (D**2, D**2)).T, k=1, which='LM')
l = np.resize(l, (D,D)).T # note transpose!

# normalise A
A = A / np.sqrt(lambdaRight)

# normalise transfer matrix
E = np.einsum('isk,jsl->ijkl', A, np.conj(A))

# normalise fixed points
norm = np.sqrt(np.einsum('ij,ji->', l, r))
l = l / norm
r = r / norm

In [5]:
assert abs(lambdaLeft - lambdaRight) < 1e-12, "Left and right fixed point values should be the same!"
assert np.allclose(l, np.einsum('ijk,li,ljm->mk', A, l, np.conj(A)), 1e-12), "l should be a left fixed point!"
assert np.allclose(r, np.einsum('ijk,kl,mjl->im', A, r, np.conj(A)), 1e-12), "r should be a right fixed point!"
assert abs(np.einsum('ij,ji->', l, r)-1) < 1e-12, "Left and right fixed points should be trace normalised!"

If we now require that there is no influence of the boundary on the bulk, we require the overlap of the boundary vectors with the fixed points to equal unity. In this case, we will have properly normalised the MPS:
![image.png](img/MPSnormalised.png)

### 1.2 Gauge fixing
While a single $A$ corresponds to a unique state $|\Psi(A)>$, the converse is not true, as different tensors may give rise to the same state. This is easily seen by noting that the gauge transform
![image.png](img/gaugeTransform.png)
leaves the physical state invariant. We may use this freedom in parametrization to impose canonical forms on the MPS tensor $A$.

We start by considering the left-orthonormal form $A_L$ of an MPS, which has the property that
![image.png](img/leftOrthonormal.png).

We can find the matrix $L$ that brings $A$ into this form by decomposing the fixed point $l$ as $l = L^\dagger L$, such that
![image.png](img/leftOrthonormal2.png)

In [6]:
"""As transfer matrix is positive definite, fixed points are also positive definite, thus allowing for a
square root algorithm of the eigenvalues. Note that taking square roots is typically less favourable,
as this increases the error"""
# l = Ldag L
S, U = np.linalg.eig(l)
L = U@np.diag(np.sqrt(S))@np.conj(U).T
# Al = L A L^-1
Al = np.einsum('ij,jkl,lm->ikm', L, A, np.linalg.inv(L))

assert np.allclose(np.einsum('ijk,ijl->kl', Al, np.conj(Al)), np.eye(D)), "Al not in left-orthonormal form"

Furthermore, there is still room for unitary gauge transformations such that we can also bring the right fixed point in diagonal form. Similarly, we can consider the right-orthonormal form $A_R$, where we have
![image.png](img/rightOrthonormal.png)

In [7]:
# r = RR
T, V = np.linalg.eig(r)
R = V@np.diag(np.sqrt(T))@np.conj(V).T
# Ar = R A R^-1
Ar = np.einsum('ij,jkl,lm->ikm', np.linalg.inv(R), A, R)

assert np.allclose(np.einsum('ijk,ljk->il', Ar, np.conj(Ar)), np.eye(D)), "Ar not in right-orthonormal form"

Finally, there is the notion of a mixed gauge for the uniform MPS by choosing a 'center site', bringing the left tensors in left-orthonormal form and the right tensors in right-orthonormal form, and redefining the center tensor as follows:
![image.png](img/mixedGauge.png)

The mixed gauge has an intuitive interpretation. Defining $C = LR$, this implements the gauge transform that maps the left-orthonormal tensor to the right-orthonormal one, hence defining the center-site tensor $A_C$:
![image.png](img/mixedGauge2.png)
The above is called the gauge condition and allows us to freely move the center tensor $A_C$ around in the MPS.

We can easily check that the above gauge fix holds for our tensors:

In [8]:
Ac = np.einsum('ij,jkl,lm->ikm', L, A, R)
C = L@R

In [9]:
LHS = np.einsum('ijk,kl->ijl', Al, C)
RHS = np.einsum('ij,jkl->ikl', C, Ar)
assert np.allclose(LHS, RHS) and np.allclose(RHS, Ac), "Something went wrong in gauging the MPS"

Finally we may perform an SVD of $C = USV^\dagger$, and taking up $U$ and $V^\dagger$ in the definition of $A_L$ and $A_R$, such that we are left with a diagonal $C$ on the virtual bonds.
![image.png](img/mixedGauge3.png)

In fact, this means that we can write down a Schmidt decomposition of the state across an arbitrary bond in the chain, and the diagonal elements $C_l$ are exactly the Schmidt numbers of any bipartition of the MPS. 
![image.png](img/SchmidtDecomp.png)
Hence, we can calculate the bipartite entanglement entropy by 
$$ S = -\sum_l C_l^2 \log(C_l^2) $$

In [10]:
# perform SVD
U, S, Vdag = svd(C)

# absorb unitaries in Al, Ar, Ac
Al = np.einsum('ij,jkl,lm->ikm', np.conj(U).T, Al, U)
Ar = np.einsum('ij,jkl,lm->ikm', Vdag, Ar, np.conj(Vdag).T)
Ac = np.einsum('ij,jkl,lm->ikm', np.conj(U).T, Ac, np.conj(Vdag).T)

# C is diagonal matrix
C = np.diag(S)

entropy = -np.sum(S**2 * np.log(S))

In [11]:
# check that gauging is still correct
LHS = np.einsum('ijk,kl->ijl', Al, C)
RHS = np.einsum('ij,jkl->ikl', C, Ar)
assert np.allclose(LHS, RHS) and np.allclose(RHS, Ac), "Something went wrong in gauging the MPS"

### 1.3 Truncation of a uniform MPS
The mixed canonical form is not only useful for finding the entanglement entropy, it also allows for a way to truncate MPS states efficiently. This is done by truncating the Schmidt decomposition, such that the new MPS has a reduced bond dimension for that ond. This truncation can be shown to be optimal in the sense that the norm between the original and the truncated MPS is minimised. The truncated MPS in the mixed gauge is thus:
![image.png](img/truncMPS.png)

We mention that this is a local optimization, in the sense that it maximizes the local overlap, however not the global overlap. This would require a variational optimization of the following cost function:
$$ || ~|\Psi(A)> - |\Psi(\tilde{A})> ||^2 $$
We postpone the detailed discussion hereof until later.

In [12]:
truncate = False
if truncate:
    # perform an SVD
    U, S, Vdag = svd(C)
    
    # truncate the SVD
    Dprime = 2
    U = U[:,:Dprime]
    Vdag = Vdag[:Dprime,:]
    S = S[:Dprime]
    
    # Absorb unitaries in Al, Ar, Ac
    Al = np.einsum('ij,jkl,lm->ikm', np.conj(U).T, Al, U)
    Ar = np.einsum('ij,jkl,lm->ikm', Vdag, Ar, np.conj(Vdag).T)
    Ac = np.einsum('ij,jkl,lm->ikm', np.conj(U).T, Ac, np.conj(Vdag).T)
    
    # C is diagonal matrix
    C = np.diag(S)
    
    # renormalise truncated MPS?
    norm = np.einsum('ijk,ijk', Ac, np.conj(Ac))
    Ac = Ac / np.sqrt(norm)
    A = Ac

# WARNING when truncated, you can no longer compare with the uniform gauge in 1.5, as A is will not be truncated
# -> use Ac as A in this case, otherwise assert checks never fullfilled

### 1.4 Algorithms for finding canonical forms
One of the great things of MPS is that they provide efficient approximations of physical states, and for gapped systems they are expected to be exact in the limit $D \rightarrow \infty$. The idea is that if we increase the bond dimension enough, we can get to arbitrary precision. However, increasing the bond dimension comes at a numerical cost, as the MPS algorithms scale in $D$. It is possible to ensure that the complexity of all MPS algorithms scales as $O(D^3)$, however this means we need to be a little smarter with the previously encountered algorithms.

As a first step, we can refrain from explicitly building the matrices that are used in the eigenvalue solvers, but this will easily extend to all following algorithms. We can circumvent explicitly building the matrix by implementing a function that corresponds to the action of the operator, usually called a handle, and passing this to the eigenvalue solver.

For example, a better way of fixing the normalisation of an MPS tensor, as well as determining left and right fixed points is by implementing the handles and using the optimal contraction sequences:

In [13]:
# calculate optimal contraction sequences
if False:
    x = np.ones(D**2)
    A = np.ones((D,d,D))
    path = np.einsum_path('ijk,ljm,km->il', A, np.conj(A), x.reshape((D, D)), optimize='optimal')
    print('optimal path for right fixed point contraction: ', path[0])
    
    path2 = np.einsum_path('ijk,ljm,li->mk', A, np.conj(A), x.reshape((D, D)), optimize='optimal')
    print('optimal path for left fixed point contraction: ', path[0])


In [14]:
def leftFixedPoint(A):
    """
    Function to determine the left fixed point of a given MPS tensor using O(D^3) algorithm
    input: A --- (D, d, D) MPStensor
    output: l --- (D, D) leftFixedPointTensor (bottom-top)
            lam --- scalar eigenvalue of leftFixedPointTensor
    """

    D = A.shape[0]

    # set optimal contraction sequence
    path = ['einsum_path', (0, 2), (0, 1)]

    # calculate transfer matrix handle and cast to LinearOperator
    transferLeftHandle = lambda v: np.reshape(
        np.einsum('ijk,ljm,li->mk', A, np.conj(A), v.reshape((D, D)), optimize=path), D ** 2)
    transferLeft = LinearOperator((D ** 2, D ** 2), matvec=transferLeftHandle)

    # calculate fixed point
    lam, l = eigs(transferLeft, k=1, which='LM')

    return lam, l.reshape(D, D)


def rightFixedPoint(A):
    """
    Function to determine the right fixed point of a given MPS tensor using O(D^3) algorithm
    input: A --- (D, d, D) MPStensor
    output: r --- (D, D) rightFixedPointTensor (top-bottom)
            lam --- scalar eigenvalue of rightFixedPointTensor
    """

    D = A.shape[0]

    # set optimal contraction sequence
    path = ['einsum_path', (0, 2), (0, 1)]

    # calculate transfer matrix handle and cast to LinearOperator
    transferRightHandle = lambda v: np.reshape(
        np.einsum('ijk,ljm,km->il', A, np.conj(A), v.reshape((D, D)), optimize=path), D ** 2)
    transferRight = LinearOperator((D ** 2, D ** 2), matvec=transferRightHandle)

    # calculate fixed point
    lam, r = eigs(transferRight, k=1, which='LM')

    return lam, r.reshape(D, D)


def fixedPoints(A):
    return leftFixedPoint(A), rightFixedPoint(A)


def normaliseFixedPoints(l, r):
    # function that normalises given left and right fixed points
    # such that they trace to unity (interpretation as density matrix)
    # returns (l, r)

    trace = np.einsum('ij,ji->', l, r)
    # trace = np.trace(rhoL*rhoR) # might work as well/be faster than einsum?
    norm = np.sqrt(trace)
    return l/norm, r/norm


def normaliseMPS(A):
    """
    Function to normalise a given MPS tensor using O(D^3) algorithm
    input: A --- (D, d, D) MPStensor
    output: A --- (D, d, D) normalised MPStensor
            l --- (D, D) leftFixedPointTensor (bottom-top)
            r --- (D, D) rightFixedPointTensor (top-bottom)
    """

    # calculate left and right fixed points of transfer matrix
    lam, l = leftFixedPoint(A)
    r = rightFixedPoint(A)[1]
    
    # normalise MPS tensor
    A /= np.sqrt(lam)
    
    # normalise fixed points
    l, r = normaliseFixedPoints(l, r)

    return A, l, r


In [15]:
A, l, r = normaliseMPS(A)
assert np.allclose(l, np.einsum('ijk,li,ljm->mk', A, l, np.conj(A)), 1e-12), "l should be a left fixed point!"
assert np.allclose(r, np.einsum('ijk,kl,mjl->im', A, r, np.conj(A)), 1e-12), "r should be a right fixed point!"
assert abs(np.einsum('ij,ji->', l, r)-1) < 1e-12, "Left and right fixed points should be trace normalised!"

Furthermore, taking the square root of such a fixed point to find the left and right gauge is also a way to introduce larger errors than neccesary. This can be circumvented by using an iterative scheme that makes use of a unique decomposition, such as a QR or polar decomposition. Note however that for a QR decomposition to be unique we need the diagonal elements of R to have a positive sign.

In [16]:
def qrPositive(A):
    """
    Function that implements a QR decomposition of a matrix A,
    such that A = QR
    input: A --- matrix (M, N)
    output: Q --- matrix (M, N), isometry
            R --- matrix (N, N), upper triangular, positive diagonal elements
    """
    
    M, N = A.shape
    
    # QR decomposition, scipy conventions: Q.shape = (M, M), R.shape = (M, N)
    Q, R = qr(A)
    
    # Throw out zeros under diagonal: Q.shape = (M, N), R.shape = (N, N)
    Q = Q[:, :N]
    R = R[:N, :]

    # extract signs and multiply with signs on diagonal
    diagSigns = np.diag(np.sign(np.diag(R)))
    Q = np.dot(Q, diagSigns)
    R = np.dot(diagSigns, R)
    
    return Q, R


def lqPositive(A):
    """
    Function that implements a LQ decomposition of a matrix A,
    such that A = LQ
    input: A --- matrix (M, N)
    output: L --- matrix (M, M), upper triangular, positive diagonal elements
            Q --- matrix (M, N), isometry   
    """
    
    M, N = A.shape
    
    # LQ decomposition: scipy conventions: Q.shape = (N, N), L.shape = (M, N)
    L, Q = rq(A)

    # Throw out zeros under diagonal: Q.shape = (M, N), L.shape = (M, M)
    Q = Q[-M:, :]
    L = L[:, -M:]

    # Extract signs and multiply with signs on diagonal
    diagSigns = np.diag(np.sign(np.diag(L)))
    Q = np.dot(diagSigns, Q)
    L = np.dot(L, diagSigns)

    return L, Q


def leftOrthonormal(A, L0=None, tol=1e-14, maxIter=1e5):
    """
    Function that brings MPS into left-orthonormal gauge,
    such that -L-A- = -Al-L-
    input: A --- MPSTensor (D, d, D)
            L0 --- initial guess for L
            tol --- convergence tolerance
            maxIter --- max amount of steps
    output: L --- Matrix (D, D) gauges A to Al
            Al --- MPSTensor (D, d, D) left-orthonormal, -conj(Al)=Al- = --
    """

    D = A.shape[0]
    d = A.shape[1]
    i = 1

    # Random guess for L0 if none specified
    if L0 is None:
        L0 = np.random.rand(D, D)

    # Normalise L0
    L0 = L0 / np.linalg.norm(L0)

    # Initialise loop
    Al, L = qrPositive(np.resize(np.einsum('ik,ksj->isj', L0, A), (D * d, D)))
    L = L / np.linalg.norm(L)
    convergence = np.linalg.norm(L - L0)

    # Decompose L*A until L converges
    while convergence > tol:
        # calculate LA and decompose
        Al, Lnew = qrPositive(np.resize(np.einsum('ik,ksj->isj', L, A), (D * d, D)))

        # normalise new L
        Lnew = Lnew / np.linalg.norm(Lnew)  # only necessary when working with unnormalised MPS?

        # calculate convergence criterium
        convergence = np.linalg.norm(Lnew - L)
        L = Lnew

        # check if iterations exceeds maxIter
        if i > maxIter:
            print("Warning, decomposition has not converged ", convergence)
            break
        i += 1

    return L, np.resize(Al, (D, d, D))


def rightOrthonormal(A, R0=None, tol=1e-14, maxIter=1e5):
    """
    Function that brings MPS into right-orthonormal gauge,
    such that -A-R- = -R-Ar-
    input: A --- MPSTensor (D, d, D)
            R0 --- initial guess for R
            tol --- convergence tolerance
            maxIter --- max amount of steps
    output: R --- Matrix (D, D) gauges A to Al
            Ar --- MPSTensor (D, d, D) right-orthonormal, -conj(Ar)=Ar- = --
    """

    D = A.shape[0]
    d = A.shape[1]
    i = 1

    # Random guess for  R0 if none specified
    if R0 is None:
        R0 = np.random.rand(D, D)

    # Normalise R0
    R0 = R0 / np.linalg.norm(R0)

    # Initialise loop
    R, Ar = lqPositive(np.resize(np.einsum('ijk,kl->ijl', A, R0), (D, D * d)))
    R = R / np.linalg.norm(R)
    convergence = np.linalg.norm(R - R0)

    # Decompose A*R until R converges
    while convergence > tol:
        # calculate AR and decompose
        Rnew, Ar = lqPositive(np.resize(np.einsum('ijk,kl->ijl', A, R), (D, D * d)))

        # normalise new R
        Rnew = Rnew / np.linalg.norm(Rnew)  # only necessary when working with unnormalised MPS

        # calculate convergence criterium
        convergence = np.linalg.norm(Rnew - R)
        R = Rnew

        # check if iterations exceeds maxIter
        if i > maxIter:
            print("Warning, decomposition has not converged ", convergence)
            break
        i += 1

    return R, np.resize(Ar, (D, d, D))


def mixedCanonical(A, L0=None, R0=None, tol=1e-14, maxIter=1e5):
    """
    Function that brings MPS into mixed gauge,
    such that -Al-C- = -C-Ar- = Ac
    input:  A --- MPSTensor (D, d, D)
            L0 --- initial guess for L
            R0 --- initial guess for R
            tol --- convergence tolerance
            maxIter --- max amount of steps
    output: Al --- MPSTensor (D, d, D) left-orthonormal, -conj(Al)=Al- = --
            Ar --- MPSTensor (D, d, D) right-orthonormal, -conj(Ar)=Ar- = --
            Ac --- MPSTensor (D, d, D) center tensor
            C --- Matrix (D, D) center matrix, -Al-C- = -C-Ar- = Ac
    """

    D = A.shape[0]

    # Random guess for  L0 if none specified
    if L0 is None:
        L0 = np.random.rand(D, D)

    # Random guess for  R0 if none specified
    if R0 is None:
        R0 = np.random.rand(D, D)
    
    # Compute left and right orthonormal forms
    L, Al = leftOrthonormal(A, L0, tol, maxIter)
    R, Ar = rightOrthonormal(A, L0, tol, maxIter)
    
    # center matrix C is matrix multiplication of L and R
    C = L @ R
    
    # singular value decomposition to diagonalise C
    U, S, Vdag = svd(C)
    C = np.diag(S)

    # absorb corresponding unitaries in Al and Ar
    Al = np.einsum('ij,jkl,lm->ikm', np.conj(U).T, Al, U)
    Ar = np.einsum('ij,jkl,lm->ikm', Vdag, Ar, np.conj(Vdag).T)

    # compute center MPS tensor
    Ac = np.einsum('ijk,kl->ijl', Al, C)
    
    # normalise MPS in mixed gauge
    nrm = np.einsum('ijk,ijk', Ac, np.conj(Ac))
    Ac /= np.sqrt(nrm);
    
    return Al, Ar, Ac, C

In [17]:
Al, Ar, Ac, C = mixedCanonical(A, L0=None, R0=None, tol=1e-14, maxIter=1e5)
assert np.allclose(np.einsum('ijk,ijl->kl', Al, np.conj(Al)), np.eye(D)), "Al1 not in left-orthonormal form"
assert np.allclose(np.einsum('ijk,ljk->il', Ar, np.conj(Ar)), np.eye(D)), "Ar not in right-orthonormal form"
LHS = np.einsum('ijk,kl->ijl', Al, C)
RHS = np.einsum('ij,jkl->ikl', C, Ar)
assert np.allclose(LHS, RHS) and np.allclose(RHS/np.sqrt(np.einsum('ijk,ijk', RHS, np.conj(RHS))), Ac), "Something went wrong in gauging the MPS"

#### Note:
For the remainder of this tutorial, keep in mind that you should always aim to reduce the complexity of your algorithm to $O(D^3)$ in order to keep the computational cost tractable.

### 1.5 Computing expectation values 
Having described the states in different gauges, we want to use these to determine the expectation values of extensive operators:
$$ O = \frac{1}{Z} \sum_{n\in Z} O_n $$

If we work with properly normalized MPS, the expectation value per site of a one-body operator (such as an order parameter e.g. $X_i$) is then found by considering the following contraction:
![image.png](img/expValue.png)

In [16]:
O = np.random.rand(d,d) + 1.0j*np.random.rand(d,d)
# convention from top to bottom

If we use the uniform gauge, we can use the fixed points of the transfer matrix to collapse everything on the left and right, such that we are left with the contraction:
![image.png](img/expValue2.png)

In [17]:
expVal = oneSiteUniform(O, A, l, r)
print(expVal)

(1.8710873507348424+1.5551861153624327j)


However, in the mixed gauge, we can locate the center site where the operator is acting, and then contract everything to the left and right to the identity, such that we find
![image.png](img/expValue3.png)

In [18]:
expValMix = oneSiteMixed(O, Ac)
print(expValMix)

(1.871087350734847+1.5551861153624247j)


In [19]:
# check if exp values are the same in different gauges
diff = abs(expVal - expValMix)
assert diff < 1e-14, "different gauges give different values?"

This procedure generalises easily to operators that act on multiple sites. In particular, the Hamiltonian $h$ can be evaluated as
![image.png](img/hamExpVal.png)
#this is not correct, the contribution where Ac stands on the right is missing, i think it is, this is the Hamiltonian density, not the complete hamiltonian so there is only a single contribution. That or i dont understand what you mean?


In [20]:
Jx, Jy, Jz, h = 1, 1, 1, 1
H = Heisenberg(Jx, Jy, Jz, h)

expVal = twoSiteUniform(H, A, l, r)
expValGauge = twoSiteMixed(H, Ac, Ar)
expValGauge2 = twoSiteMixed(H, Al, Ac)

print(expVal)
print(expValGauge)
print(expValGauge2)
diff1 = abs(expVal - expValGauge)
diff2 = abs(expVal - expValGauge2)

assert diff1 < 1e-14 and diff2 < 1e-14, "different gauges give different values?"

(-0.9878470797955999-5.421010862427522e-17j)
(-0.9878470797955928-7.922368163212073e-17j)
(-0.9878470797955954+1.0845550519353461e-17j)


### 1.6 Summary